In [ ]:
import torch
torch.__version__

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import sys
sys.  version

In [ ]:
print('is there cuda available:',torch.cuda.is_available())
print('working of gpu',torch.cuda.current_device())
print('Name of gpu original or scientific',torch.cuda.get_device_name(0))

In [ ]:
#want to change the gpu
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
#Import basic libraries
import pandas as pd
import numpy as np

In [ ]:
# If you want to install the packages
# install pytorch-pretrained-bert from PyPI
#!pip install pytorch-pretrained-bert

In [ ]:
#All important libraries to run 
import os
import math
import random
import csv
import sys

from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
import statistics as stats

#Cloned the bert_sklearn API here
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

#Import the pretrained model 
sys.path.append("../") 
from bert_sklearn import BertTokenClassifier, BertClassifier, BertRegressor
#It is use to load the trained model
from bert_sklearn import load_model

In [ ]:
#data load of pet
df_pet = pd.read_csv(path + '2_abstract_pet_recycle_methods.csv')
df_pet =  df_pet.drop(['Unnamed: 0'], axis=1)
print('Shape of data frame:', df_pet.shape)
df_pet.head()

In [ ]:
#data load of ps
df_ps = pd.read_csv(path + '0_abstract_ps_recycle_methods.csv')
df_ps =  df_ps.drop(['Unnamed: 0'], axis=1)
print('Shape of data frame:', df_ps.shape)
df_ps.head()

In [ ]:
# #data load of pe
df_pe = pd.read_csv(path + '2_abstract_pe_recycle_methods.csv')
df_pe =  df_pe.drop(['Unnamed: 0'], axis=1)
print('Shape of data frame:', df_pe.shape)
df_pe.head()

In [ ]:
#data load of pp
df_pp = pd.read_csv(path + '2_abstract_pp_recycle_methods.csv')
df_pp =  df_pp.drop(['Unnamed: 0'], axis=1)
print('Shape of data frame:', df_pp.shape)
df_pp.head()

In [ ]:
# concatenating df1 and df2 along rows
df_net = pd.concat([df_pe, df_pet,df_pp, df_ps], axis=0).reset_index(drop = True)
print('Net shape of data frame:', df_net.shape)

In [ ]:
df_rel = pd.read_csv(path + 'elsevier_data.csv')
df_rel =  df_rel.drop(['Unnamed: 0',"DOI",'Title'], axis=1)
print('Shape of data frame:', df_rel.shape, df_rel.columns)
df_rel.head()

In [ ]:
#Relevant abstracts labelling
df_net['label'] = 0

In [ ]:
#Delete the all entries with nan in abstract column
NO_INDEX = []
for i in range(len(df_net)):
    if type(df_net.Abstract[i]) != float: 
        NO_INDEX.append(i)
#Select some entries        
df_net = df_net.iloc[NO_INDEX].reset_index(drop = True)
print('Shape of data frame before:', df_net.shape)

#Delete the all entries with integer in abstract column
NO_INDEXX = []
for i in range(len(df_net)):
    if df_net.Abstract[i] != '0': 
        NO_INDEXX.append(i)
#Select some entries        
df_net = df_net.iloc[NO_INDEXX].reset_index(drop = True)
print('Shape of data frame after:', df_net.shape)

df_net.head()

In [ ]:
# data load of pp
df_h2 = pd.read_csv(path + 'elsevier_abstract.csv')
df_h2 =  df_h2.drop(['Unnamed: 0'], axis=1)
print('Shape of data frame:', df_h2.shape)
df_h2.head()

In [ ]:
#data load of pp
df_m2 = pd.read_csv(path + 'methanol_synthesis.csv')
df_m2 =  df_m2.drop(['Unnamed: 0',"DOI",'Title'], axis=1)
print('Shape of data frame:', df_m2.shape)
df_m2.head()

In [ ]:
# # concatenating df1 and df2 along rows
df_net_1 = pd.concat([df_h2,df_m2], axis=0).reset_index(drop = True)
print('Net shape of data frame:', df_net_1.shape)

In [ ]:
#Relevant abstracts labelling
df_net_1['label'] = 1  #Not relevent

In [ ]:
# concatenating df1 and df2 along rows
df_net_total = pd.concat([df_net_1, df_net], axis=0).reset_index(drop = True)
print('Net shape of data frame:', df_net_total.shape)

In [ ]:
#Delete the all entries with nan in abstract column
NO_INDEX = []
for i in range(len(df_net_total)):
    if type(df_net_total.Abstract[i]) != float: 
        NO_INDEX.append(i)
#Select some entries        
df_net_total = df_net_total.iloc[NO_INDEX].reset_index(drop = True)
print('Shape of data frame before:', df_net_total.shape)

#Delete the all entries with integer in abstract column
NO_INDEXX = []
for i in range(len(df_net_total)):
    if df_net_total.Abstract[i] != '0': 
        NO_INDEXX.append(i)
#Select some entries        
df_net_total = df_net_total.iloc[NO_INDEXX].reset_index(drop = True)
print('Shape of data frame after:', df_net_total.shape)

df_net_total.head()

In [ ]:
#Abstract cleaning
def deletespace(text):
    text0 = text.replace("<jats:p>","" )
    text1 =  text0.replace("</jats:p>", '')
    text2 = text1.replace('<jats:title>Abstract</jats:title>\n', '')
    text3 = text2.replace('<p>', "")
    text4 = text3.replace('</p>', "")
    text5 = text4.replace('\n', "")
    
    return text5
df_net_total['Abstract'] = df_net_total['Abstract'].apply(deletespace)

In [ ]:
df =df_net_total

In [ ]:
#Remove the punctutions from each sentence 
sentence = []
import re
for i in np.arange(0,df.shape[0], 1):
    pattern = r'\[[^()]*\]'
    #s = """Issachar is a rawboned[a] (donkey) lying down among the sheep pens."""
    t = re.sub(pattern, '', df.Abstract[i])
    sentence.append(t)
df['sentence'] = sentence
df.head()

In [ ]:
#The data has input and output
X= df.sentence
Y = df.label

In [ ]:
#The data is splited into train and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3, random_state= 3)

In [ ]:
#Total data points describe
print('shape of actual data', df.shape[0])
print('shape of train data', X_train.shape[0])
print('shape of test data', X_test.shape[0])

In [ ]:
#Load the pretrained model of BERTclassifier
%%time
model = BertClassifier(bert_model='bert-base-cased',
                            max_seq_length=256,
                            epochs=3,
                            gradient_accumulation_steps=2,
                            learning_rate=3e-6,
                            train_batch_size=32,
                            eval_batch_size=32,
                            validation_fraction=0.30,
                       logfile='bert_sklearn_new.log',
                      warmup_proportion = 0.2,
                      random_state = 42)

print(model)

In [ ]:
# finetune model on train data
model.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1,len(train_loss)+1,1), train_loss, color='red', label= 'Train', linewidth=2.5)
plt.plot(range(1,len(val_loss)+1,1), val_loss, color =  'Blue', label = 'Validation', linewidth=2.5)
#plt.title('Classification loss')
plt.legend(frameon=False,prop={'weight':'bold',"size":16})
plt.xlabel('Number of epoch',fontweight='bold', fontsize=18)
plt.ylabel('Loss',fontweight='bold', fontsize=18)
#plt.title('GP Regression',fontweight='bold')
#plt.axis('square')
from matplotlib import rc

plt.rcParams['axes.linewidth'] = 3

plt.tick_params(axis="x", direction="in",width=4)
plt.tick_params(axis="y", direction="in", width=4)

rc('font', weight='bold')

plt.tick_params(bottom=True, top=False, left=True, right=False)
plt.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
plt.xticks(rotation = '0', fontsize = 14)
plt.yticks(rotation = '0', fontsize = 14)
#plt.legend(handles=h, labels=np.arange(0.9,0.85), title="Quality")
#plt.rcParams.update({'legend.fontweight':'bold'}
#plt.savefig('line_plot.pdf')
plt.savefig(r'learning_classi.pdf', dpi=5000,bbox_inches ="tight")
plt.show()

In [ ]:
#Save model to disk
savefile = 'classifi_model50_256_with_netrual.bin'
model.save(savefile)

In [ ]:
# get predictions on test data
y_pred_train = model.predict(X_test)

# get predictions on train data
y_pred_test = model.predict(X_train)

In [ ]:
# print report on classifier stats
print(classification_report(df_net_total.label, y_pred_train))

#Get accuracy on train data
print('Accuracy of test:', accuracy_score(df_net_total.label, y_pred_train))

In [ ]:
#Print report on classifier stats
print(classification_report(y_test, y_pred_test))

#Accuracy on train data
print('Accuracy of test:', accuracy_score(y_test, y_pred_test))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Change figure size and increase dpi for better resolution
plt.figure(figsize=(8,6), dpi=100)
# Scale up the size of all text
sns.set(font_scale = 1.1)

# Plot Confusion Matrix using Seaborn heatmap()
# Parameters:
# first param - confusion matrix in array format   
# annot = True: show the numbers in each heatmap cell
# fmt = 'd': show numbers as integers. 
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_test)

ax = sns.heatmap(confusion_matrix, annot=True, fmt='d',square=True)

# set x-axis label and ticks. 
ax.set_xlabel("Predicted Class", fontsize=22,labelpad=20, fontweight='bold')
ax.xaxis.set_ticklabels(['0', '1'], fontsize=20, fontweight='bold')

# set y-axis label and ticks
ax.set_ylabel("Actual Class", fontsize=22, labelpad=20, fontweight='bold')
ax.yaxis.set_ticklabels(['0', '1'], fontsize=20, fontweight='bold')

# set plot title
#ax.set_title("Confusion Matrix for the Diabetes Detection Model", fontsize=14, pad=20)
#plt.savefig("test_confusion.pdf",dpi=2000,bbox_inches='tight')

plt.show()

**End of code**